In [2]:
%pip install -r requirements.txt

  Using cached numpy-2.0.2-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp311-cp311-macosx_14_0_arm64.whl (5.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.0.2 which is incompatible.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.2 which is incompatible.
grc-proiel-trf 3.7.5 requires spacy<3.8.0,>=3.7.5, but you have spacy 3.8.2 which is incompatible.
grc-proiel-sm 3.7.5 requires spacy<3.8.0,>=3.7.5, but you have spacy 3.8.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -

In [4]:
%pip install --upgrade numpy==1.24.3


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%run -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.7.1
    Uninstalling en-core-web-sm-3.7.1:
      Successfully uninstalled en-core-web-sm-3.7.1
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [30]:
from pathlib import Path

import pandas as pd
import spacy

f = Path(f"./tlg0525.tlg001.perseus-eng2.pickle")

def tokenize(df: pd.DataFrame):
    model = "en_core_web_sm"

    nlp = spacy.load(model, disable=["ner"])

    df["tokens"] = df["unannotated_strings"].apply(nlp.tokenizer)

    raw_texts = [t for t in df["unannotated_strings"]]
    annotated_texts = nlp.pipe(raw_texts, batch_size=100)

    df["nlp_docs"] = list(annotated_texts)

    return df

df = tokenize(pd.read_pickle(f))

# Bug 1

Why is the function `expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1)` returning `0.0` no matter what we pass to it?

In [50]:
from collections import Counter

def expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1):
    """
    `node` and `collocate` should be the string representations
    of the associated lemmata
    """
    # The following (commented out) line was causing the issues because it  
    # attempted to use the Doc object directly, which doesn't provide the individual 
    # words. By changing it to t.lemma_, you access the lemmatized form of each 
    # token in the Doc, allowing you to collect the correct base forms of the words 
    # for the analysis. 

    # lemmata = [t for t in df['nlp_docs'].explode()]
    lemmata = [t.lemma_ for t in df['nlp_docs'].explode()]

    counter = Counter(lemmata)
    node_count = counter[node]
    collocate_count = counter[collocate]

    return (node_count * collocate_count * window_size) / len(lemmata)

expected_freq_the_mainlaind = expected_frequency_of_collocation(df, "mainland", "the")

expected_freq_the_mainlaind

3.2848751741271176

# Bug 1 analysis and response

# Bug 2

Why does the code below not give us the number of occurrences of the token "the"?

In [49]:
# The original code observed_freq_the = df['tokens'].str.find('the') was attempting 
# to find the index of the substring "the" in each entry of the tokens column, 
# which contains spaCy Doc objects, rather than string values. As a result, it 
# returned NaN because the str.find() method is designed for string operations, 
# not for Doc objects. By using observed_freq_the = df['observed_freq_the'].sum(),
# I aggregated the total counts of occurrences of "the" across all documents.

# observed_freq_the = df['tokens'].str.find('the')

observed_freq_the = df['observed_freq_the'].sum()
observed_freq_the

26932

# Bug 2 analysis and response

# Bug 3

How is `o11` greater than `r1`, and why is our `o12` negative?

In [51]:
node = "mainland"
collocate = "the"

# The updated count_ngram_collocations function now identifies the indexes of the 
# target word within the token list, which allows it to check for the presence of
#  the collocate within a specified window around each occurrence of w1. By using a 
# fixed window size with defined left and right bounds, the function ensures that 
# it counts each unique co-occurrence of w1 and w2 correctly, preventing overlaps 
# and ensuring accurate counts. This approach resolves the issue where the previous
#  counting method might have mistakenly counted multiple collocations for the same 
# instance of w1.

# def count_ngram_collocations(x, w1, w2, l_size: int = 1, r_size: int = 1):
#     lemmata = [t.lemma_ for t in x]

#     # the right-hand side of a slice in Python is exclusive, so we add 1 to make sure
#     # we're actually getting one element to the right

#     chunked_lemmata = [lemmata[i - l_size:i + r_size + 1] for i in range(0, len(lemmata))]
    
#     cooccurrences = [1 for l in chunked_lemmata if w1 in l and w2 in l]

#     return sum(cooccurrences)

def count_ngram_collocations(x, w1, w2, l_size: int = 2, r_size: int = 1):
    lemmata = [t.lemma_ for t in x]
    indexes = [i for i, lemma in enumerate(lemmata) if lemma == w1]
    cooccurrences = 0
    for i in indexes:
        left = max(i - l_size, 0)
        right = min(i + r_size + 1, len(lemmata))

        window = lemmata[left:right]

        if w2 in window:
            cooccurrences += 1
    return cooccurrences

r1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == node])
r2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != node])
c1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == collocate])
c2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != collocate])

o11 = df["nlp_docs"].apply(count_ngram_collocations, args=(node, collocate)).sum()
o12 = r1 - o11
o21 = c1 - o11

print(f"r1: {r1}, r2: {r2}, c1: {c1}, c2: {c2}, o11: {o11}, o12: {o12}, o21: {o21}")

r1: 38, r2: 311516, c1: 26932, c2: 284622, o11: 34, o12: 4, o21: 26898


# Bug 3 analysis and response